# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/justinasmarozas/OneDrive/Udacity/Data Engineer NanoDegree/github/data-engineer-nanodegree/2. Data Modelling/projects/2. Data Modelling with Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow([
        'artist',
        'firstName',
        'gender',
        'itemInSession',
        'lastName',
        'length',
        'level',
        'location',
        'sessionId',
        'song',
        'userId'
    ])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((
            row[0], 
            row[2], 
            row[3], 
            row[4], 
            row[5], 
            row[6], 
            row[7], 
            row[8], 
            row[12], 
            row[13], 
            row[16]
        ))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
command = """
create keyspace if not exists udacity
with replication = {
    'class': 'SimpleStrategy',
    'replication_factor' : 1
};
"""
try:
    session.execute(command)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
# sessionId = 338, and itemInSession = 4

query_1 = """
select
    artist,
    song,
    length,
    userId,
    sessionId,
    itemInSession
from UserSessionItems
where userId = 50
and sessionId = 338
and itemInSession = 4;
"""

In [9]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

query_2 = """
select
    artist,
    song,
    firstName,
    lastName
from UserSessionItems
where userId = 10
and sessionId = 182
order by
    sessionId,
    itemInSession;
"""

In [10]:
# Query 3: Give me every user name (first and last) in my music app history who listened 
# to the song 'All Hands Against His Own'

query_3 = """
select
    firstName,
    lastName
from SongListens
where song = 'All Hands Against His Own';
"""                 

### Create tables

Each row can be identified by a combination of `sessionId` and `itemInSession` columns. These columns will be required in all primary keys to avoid data loss. Other columns used in primary keys are there to enable specific queries.

Queries 1 and 2 are similar in nature. I'll use a single table for them. I acknowledge that query 1 would be slightly simpler if separate tables were used but at this time I'll be using one.

Query 3 will need a separate table.

In [11]:
user_session_items_ddl = """
create table if not exists UserSessionItems
(
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
    firstName text,
    lastName text,
    primary key (userId, sessionId, itemInSession)
);
"""

try:
    session.execute(user_session_items_ddl)
except Exception as e:
    print(e)

In [12]:
song_listens_ddl = """
create table if not exists SongListens
(
    song text,
    sessionId int,
    itemInSession int,
    firstName text,
    lastName text,
    primary key (song, sessionId, itemInSession)
);
"""

try:
    session.execute(song_listens_ddl)
except Exception as e:
    print(e)

### Fill tables with data

In [13]:
def insert_user_session_item(line):
    command = """
insert into UserSessionItems (
    userId,
    sessionId,
    itemInSession,
    artist,
    song,
    length,
    firstName,
    lastName
)
values (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    session.execute(
        command, (
            int(line[10]), 
            int(line[8]), 
            int(line[3]), 
            line[0], 
            line[9], 
            float(line[5]), 
            line[1], 
            line[4],
        )
    )

In [14]:
def insert_song_listen(line):
    command = """
insert into SongListens (
    song,
    sessionId,
    itemInSession,
    firstName,
    lastName
)
values (%s, %s, %s, %s, %s);
    """
    session.execute(
        command, (
            line[9], 
            int(line[8]), 
            int(line[3]), 
            line[1], 
            line[4],
        )
    )

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_user_session_item(line)
        insert_song_listen(line)

### Run queries to verify that the data have been inserted into each table

In [16]:
try:
    rows = session.execute(query_1)
    print(f'Row count: {len(rows.current_rows)}.')
    print(f'Columns: {rows.column_names}.')
except Exception as e:
    print(e)

for row in rows:
    print((row.userid, row.sessionid, row.iteminsession, row.artist, row.song, row.length))

Row count: 1.
Columns: ['artist', 'song', 'length', 'userid', 'sessionid', 'iteminsession'].
(50, 338, 4, 'Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875)


In [17]:
try:
    rows = session.execute(query_2)
    print(f'Row count: {len(rows.current_rows)}.')
    print(f'Columns: {rows.column_names}.')
except Exception as e:
    print(e)

for row in rows:
    print((row.artist, row.song, row.firstname, row.lastname))

Row count: 4.
Columns: ['artist', 'song', 'firstname', 'lastname'].
('Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz')
('Three Drives', 'Greece 2000', 'Sylvie', 'Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz')


In [18]:
try:
    rows = session.execute(query_3)
    print(f'Row count: {len(rows.current_rows)}.')
    print(f'Columns: {rows.column_names}.')
except Exception as e:
    print(e)

for row in rows:
    print((row.firstname, row.lastname))

Row count: 3.
Columns: ['firstname', 'lastname'].
('Sara', 'Johnson')
('Jacqueline', 'Lynch')
('Tegan', 'Levine')


### Drop the tables before closing out the sessions

In [19]:
commands = [
    "drop table if exists UserSessionItems;",
    "drop table if exists SongListens;"
]

try:
    for command in commands:
        session.execute(command)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [20]:
session.shutdown()
cluster.shutdown()